# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introduction-of-Functional-API" data-toc-modified-id="Introduction-of-Functional-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction of Functional API</a></div><div class="lev2 toc-item"><a href="#Example-1" data-toc-modified-id="Example-1-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Example-1</a></div><div class="lev2 toc-item"><a href="#Simple-Concatenation-Neural-Network-Example" data-toc-modified-id="Simple-Concatenation-Neural-Network-Example-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Simple Concatenation Neural Network Example</a></div>

# Introduction of Functional API

In [1]:
from tensorflow.keras import Input, layers

In [2]:
input_tensor = Input(shape=(32,)) # A tensor

In [3]:
dense = layers.Dense(32, activation='relu') # A layer is a function

In [4]:
output_tensor = dense(input_tensor) # A layer inputs a tensor and returns a tensor

2021-11-05 05:37:40.454005: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Example-1

In [5]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import Input

Peviously, how we built our sequential model

In [6]:
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

The equivalent way, use layer as function

In [7]:
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

In [8]:
model = Model(input_tensor, output_tensor) # The Model class turns an input tensor and output tensor into a "model" variable

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


The only part that may seem a bit magical at this point is instantiating a Model object using only an input tensor and an output tensor. Behind the scenes, Keras retrieves every layer involved in going from input_tensor to output_tensor, bringing them together into a graph-like data structure—a Model. Of course, the reason it works is that output_tensor was obtained by repeatedly transforming input_tensor. 

<b> If you tried to build a model from inputs and outputs that weren’t related, you’d get a RuntimeError: </b>

In [10]:
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input, output_tensor)

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'") at layer "dense_4". The following previous layers were accessed without issue: []

In [12]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))
model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)

2021-11-05 05:37:54.549284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
8/8 [==============================] - 0s 1ms/step - loss: 11.8819
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 12.9022
Epoch 3/10
8/8 [==============================] - 0s 902us/step - loss: 14.4794
Epoch 4/10
8/8 [==============================] - 0s 1ms/step - loss: 16.4463
Epoch 5/10
8/8 [==============================] - 0s 768us/step - loss: 18.4872
Epoch 6/10
8/8 [==============================] - 0s 1ms/step - loss: 20.4081
Epoch 7/10
8/8 [==============================] - 0s 877us/step - loss: 22.5551
Epoch 8/10
8/8 [==============================] - 0s 1ms/step - loss: 25.2627
Epoch 9/10
8/8 [==============================] - 0s 1ms/step - loss: 28.4978
Epoch 10/10
32/32 [==============================] - 0s 626us/step - loss: 34.1941


## Simple Concatenation Neural Network Example

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

In [28]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500
# note, we didn't look for the top words, meaning, the top_words = text_vocabulary_size or question_vocabulary_size, since, if we set
# top_words = 100, for example, then our inputs
# text = np.random.randint(1, text_vocabulary_size, 
#                         size=(num_samples, max_length))
# may generate some words that are not those top 100 words, thus run into error.    

text_input = Input(shape=(None,), dtype='int32', name='text')
#embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input) # There is a bug in the book's code, should switch the order of arguments

embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input) # There is a bug in the book's code, should switch the order of arguments
encoded_text = layers.LSTM(32)(embedded_text)


question_input = Input(shape=(None,),
                       dtype='int32',
                       name='question')

#embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input) # There is a bug in the book's code, should switch the order of arguments

embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input) # There is a bug in the book's code, should switch the order of arguments
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1) # concat the outputs of two LSTMs, the hidden states. with shape (None, 32) and (None, 16)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)



In [29]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

Now, how do you train this two-input model? There are two possible APIs: you can feed the model a list of Numpy arrays as inputs, or you can feed it a dictionary that maps input names to Numpy arrays. Naturally, the latter option is available only if you give names to your inputs.

In [30]:
import numpy as np
num_samples = 1000
max_length = 100
text = np.random.randint(1, text_vocabulary_size, 
                         size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, 
                             size=(num_samples, max_length))
answers = np.random.randint(0, 1, 
                            size=(num_samples, answer_vocabulary_size))

In [31]:
#model.fit([text, question], answers, epochs=10, batch_size=128)
# output will be meaningless, since we are not training meaningful inputs/outputs.
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 2s 48ms/step - loss: 0.0000e+00 - acc: 0.0010
Epoch 2/10
8/8 [==============================] - 0s 47ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 0s 45ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 0s 47ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 0s 50ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 0s 48ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 0s 45ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 8/10
8/8 [==============================] - 0s 50ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 9/10
8/8 [==============================] - 0s 47ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 10/10
8/8 [==============================] - 0s 47ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
